In [82]:
from nsepy import get_history
import pandas as pd
import numpy as np
import datetime
from datetime import date
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [83]:
stock_name = input('Enter NSE Stock Name: ').upper()
date_entry = input('Enter a date in yyyy,m,d format')
year, month, day = map(int, date_entry.split(','))
start_date= datetime.date(year, month, day)
end_date = date.today()
# get_history(symbol = 'SBIN', start=date(2015,1,1), end=date(2022,3,1))
def stock_df(s, sd, ed):
    df = get_history(symbol=s, start=sd, end=ed)
    return df

In [84]:
df = stock_df(stock_name, start_date, end_date)
df.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2015-01-01,SBIN,EQ,311.85,312.45,315.00,310.70,314.0,314.00,313.67,6138488,1.925489e+14,58688,1877677,0.3059
2015-01-02,SBIN,EQ,314.00,314.35,318.30,314.35,315.6,315.25,316.80,9935094,3.147389e+14,79553,4221685,0.4249
2015-01-05,SBIN,EQ,315.25,316.25,316.80,312.10,312.8,312.75,313.84,9136716,2.867432e+14,88236,3845173,0.4208
2015-01-06,SBIN,EQ,312.75,310.00,311.10,298.70,299.9,299.90,305.14,15329257,4.677601e+14,169268,7424847,0.4844
2015-01-07,SBIN,EQ,299.90,300.00,302.55,295.15,301.4,300.15,299.95,15046745,4.513243e+14,147185,5631400,0.3743


In [85]:
df = df['Close']
df.head()

Date
2015-01-01    314.00
2015-01-02    315.25
2015-01-05    312.75
2015-01-06    299.90
2015-01-07    300.15
Name: Close, dtype: float64

In [86]:
px.line(df)

In [87]:
df=df.reset_index()['Close']
df.head(), df.shape

(0    314.00
 1    315.25
 2    312.75
 3    299.90
 4    300.15
 Name: Close, dtype: float64,
 (1827,))

In [88]:
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]
train_data.shape,test_data.shape

((1461,), (366,))

In [89]:
# Normalizing the data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data.values.reshape(-1,1))
test_data = scaler.transform(test_data.values.reshape(-1,1))
train_data.shape, test_data.shape

((1461, 1), (366, 1))

In [90]:
train_data

array([[0.73640262],
       [0.74204469],
       [0.73076055],
       ...,
       [0.41480478],
       [0.39652449],
       [0.4197698 ]])

In [91]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [92]:
X_train, y_train = create_dataset(train_data, 50)
X_test, y_test = create_dataset(test_data, 50)

In [93]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1410, 50), (1410,), (315, 50), (315,))